In [118]:
import os
import datetime
import time

import ipywidgets as widgets

import IPython
import IPython.display
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import clear_output

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

from math import sqrt
from numpy import concatenate
import numpy as np
import math

import pandas as pd
from pandas import DataFrame
from pandas import concat

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

# import tensorflow 
# import tensorflow as tf
# from tensorflow import keras

import platform
# print(platform.python_version())
# print(tf.version.VERSION)
# print(np.__version__)



In [119]:
# File name and Path
cwd = os.getcwd()
path_adress = "/Import_Dataset/"

# ข้อมูล พิกัดสถานที่เเละรายชื้ออ้างอิงสถานที่ตรวจวัด
file_name_st = 'Data_lat_long_Rice research Center'
csv_file_st = cwd + path_adress + file_name_st + '.csv'

df_st = pd.read_csv(csv_file_st)
print("All low RiceCenter {} station" .format(df_st.shape[0]))

data_list_st_num = list(range(1, len(df_st)))
data_list_st_name = []
data_list_st_name = df_st['nameEng'].values.tolist()
data_list_st_lat = df_st['Latitude'].values.tolist()
data_list_st_long = df_st['Longitude'].values.tolist()

All low RiceCenter 34 station


In [120]:
def creat_avrdata(frames_sma,rolling_num):
    
    h_name = list(frames_sma)
    frames_sma['mirid bug'] = frames_sma['mirid bug'].rolling(rolling_num, min_periods=1).sum()
    frames_sma['bph'] = frames_sma['bph'].rolling(rolling_num, min_periods=1).sum()
    for i in range(7,14):
        frames_sma[h_name[i]] = frames_sma[h_name[i]].rolling(rolling_num, min_periods=1).mean()
#        print(header_name[i])

    #defining rate
    our_rate = rolling_num
    #apply the rate
    constant_subset = frames_sma[::our_rate] 
        
    frames = constant_subset
    return frames


In [128]:
st = 1
# st_num = 1
file_name = df_st['nameEng'][data_list_st_num[st]]
name_input = file_name
name_locals = 'stN_' + file_name
print(name_locals)

# File name and Path
csv_file = cwd + path_adress + name_input + '.csv'
# index_col=0 , index_col=None
dataset = pd.read_csv(csv_file, header=0,
                      index_col=0, encoding="TIS-620")

header_name = list(dataset)
# print(header_name)

data_list_header_num = list(range(0, len(header_name)))
# print(data_list_header_num)

data_list_drop = [0,4,5]

for i in range(len(data_list_drop)):
        dataset = dataset.drop([header_name[data_list_drop[i]]], axis=1)

dataset.head()


stN_Chai Nat Rice Seed Center 1


,latitude,longitude,day,mirid bug,mint,maxt,temp,dew,humidity,wspd,...,R-12,R-13,R-14,R-15,R-16,R-17,R-18,R-19,R-20,bph
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,15.348,99.597,1,1,24.8,29.2,27.3,10.9,36.44,23.6,...,0,0,12527,182,1142,807,1324,1544,560,2
2015-01-02,15.348,99.597,2,3,22.3,27.6,25.0,10.8,42.21,13.2,...,0,0,12527,182,1142,807,1324,1544,560,14
2015-01-03,15.348,99.597,3,1,24.3,29.9,27.7,13.9,43.33,7.5,...,0,0,12527,182,1142,807,1324,1544,560,4
2015-01-04,15.348,99.597,4,0,25.5,30.3,27.9,15.0,45.95,4.8,...,0,0,12527,182,1142,807,1324,1544,560,6
2015-01-05,15.348,99.597,5,8,26.4,26.4,26.4,18.6,62.34,0.0,...,0,0,12527,182,1142,807,1324,1544,560,18


In [130]:
# dataset
frames_train = creat_avrdata(dataset,7)
frames_train.head()
# len(frames_train)

,latitude,longitude,day,mirid bug,mint,maxt,temp,dew,humidity,wspd,...,R-12,R-13,R-14,R-15,R-16,R-17,R-18,R-19,R-20,bph
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,15.348,99.597,1,1.0,24.8,29.2,27.3,10.900000,36.440000,23.600000,...,0,0,12527,182,1142,807,1324,1544,560,2.0
2015-01-08,15.348,99.597,8,79.0,25.8,26.5,26.2,13.761497,44.906065,11.255408,...,0,0,12527,182,1142,807,1324,1544,560,276.0
2015-01-15,15.348,99.597,15,114.0,23.0,23.0,23.0,17.844898,63.125510,7.167347,...,0,0,12527,182,1142,807,1324,1544,560,427.0
2015-01-22,15.348,99.597,22,124.0,25.2,30.7,27.9,13.883673,50.701429,7.644898,...,0,0,12527,182,1142,807,1324,1544,560,540.0
2015-01-29,15.348,99.597,29,55.0,25.8,32.0,28.9,16.759184,51.427143,4.971429,...,0,0,12527,182,1142,807,1324,1544,560,347.0


In [ ]:
# frames_train.head()
dataset.head()

In [ ]:
# df_temperature = dataset['temp']
df_temperature = pd.DataFrame(dataset['bph'].loc['2015-01-01':'2015-01-31']) 

# df_temperature = 
# the simple moving average over a period of 10 years
# df_temperature['7D-SMA'] = df_temperature.bph.rolling(7, min_periods=1).mean()
# df_temperature['3D-SMA'] = df_temperature.bph.rolling(3, min_periods=1).mean()
df_temperature['7D-SMA'] = df_temperature.bph.rolling(7, min_periods=1).sum()
df_temperature['3D-SMA'] = df_temperature.bph.rolling(3, min_periods=1).sum()
# df_temperature

In [ ]:
# colors for the line plot
colors = ['green', 'red', 'purple']

# line plot - the yearly average air temperature in Barcelona
df_temperature.plot(color=colors, linewidth=3, figsize=(12,6))

# modify ticks size
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(labels =['Data Input', '10-years SMA', '20-years SMA'], fontsize=14)

# title and labels
plt.title('Data average ', fontsize=18)
plt.xlabel('date-time', fontsize=16)
plt.ylabel('Temperature [°C]', fontsize=16)

In [ ]:
#defining rate
our_rate = 6 
#apply the rate
constant_subset = df_temperature[::our_rate] 
#data
constant_subset

In [ ]:
def creat_dataset(st_BPH=0, y_1='2015', y_2='2019', mode=1):
    for i in range(len(data_list_st_num)):
        st = i
        file_name = df_st['nameEng'][data_list_st_num[st]]
        name_input = file_name
        name_locals = 'stN_' + file_name

        # File name and Path
        csv_file = cwd + path_adress + name_input + '.csv'
        # index_col=0 , index_col=None
        dataset = pd.read_csv(csv_file, header=0,
                              index_col=0, encoding="TIS-620")
        dataset = dataset.drop(['address'], axis=1)
        dataset = dataset.drop(['year'], axis=1)
        dataset = dataset.drop(['dew'], axis=1)
        # dataset = dataset.drop(['latitude'], axis=1)
        # dataset = dataset.drop(['longitude'], axis=1)
        dataset.columns
        locals()[name_locals] = dataset
        # print(f'Dataframe name_station: {st+1 , name_input}')
        print('wait......')
        del dataset
        clear_output(wait=True)

    date_start = y_1 + '-01' + '-01'
    date_stop = y_2 + '-12' + '-31'

#---------------------------------------------------------------------------------------------------------------------------------------------------#
    if mode == 0:
        file_name = df_st['nameEng'][st_BPH]
        locals_input = 'stN_' + file_name
        print(locals_input)
        dataset_st = locals()[locals_input].loc[date_start:date_stop]
        frames_st = dataset_st
    else:
        m = 0
        for j in range(len(data_list_st_num)):
            # for j in range(df_st.shape[0]):
            # file_name = df_st['nameEng'][j]
            file_name = df_st['nameEng'][data_list_st_num[j]]
            locals_input = 'stN_' + file_name
            print(locals_input)
            dataset_st = locals()[locals_input].loc[date_start:date_stop]
            clear_output(wait=True)
            if m == 0:
                frames_st = dataset_st
                m = m+1
                print(m)
            else:
                frames_st = [frames_st, dataset_st]
                frames_st = pd.concat(frames_st)
#---------------------------------------------------------------------------------------------------------------------------------------------------#
    # frames=frames_st[new_colum_2]
    frames = frames_st
    return frames


In [ ]:
frames_train = creat_dataset(0,'2015','2019',1)

df = frames_train.reset_index()
date_time_predict = pd.to_datetime(df.pop('date'))

# date_time_predict
df

In [ ]:
# frames_train
our_rate = 6 

#conditonal sampling
our_condition = df['latitude'] == 15.198

#Retirive the index
index = our_condition[our_condition == True].index
 
#sample based on condition 
conditional_subset = df[our_condition][::our_rate] 
 
#output 
df2 = conditional_subset.set_index()
df2

In [ ]:
our_condition

In [ ]:
#defining rate
our_rate = 6 
#apply the rate
constant_subset = df_temperature[::our_rate] 
#data
constant_subset